In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
country='Israel'   #Selected Country
countryConf='Israel'   #Selected Country

#reading dataset covid19 confirm cases
dfcases= pd.read_csv("../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_confirmed_cases.csv")

#copy from the selected country
dfcases = dfcases[['Country/Region',countryConf]].copy()
dfcases

In [ ]:
dfcases.columns=['date','cases']           #rename columns
dfcases =dfcases.drop([0])                 #drop the first row

#convert object  to date
dfcases['date'] =  pd.to_datetime(dfcases['date'], format='%m/%d/%y')
dfcases

In [ ]:
#reading dataset vaccination progress
dfvac= pd.read_csv("../input/covid-world-vaccination-progress/country_vaccinations.csv")

#copy date and data from selected country
dfvac = dfvac.loc[dfvac['country']==country]
dfvac

In [ ]:
#convert object to date
dfvac['date'] =  pd.to_datetime(dfvac['date'], format='%Y-%m-%d')

#merge dataset vaccination progress and dataset from cases in Israel
df = pd.merge(dfcases,dfvac, how='left' , left_on='date', right_on='date')
df

In [ ]:
#copy date, cases, people_vaccinated,people_fully_vaccinated
df = df[['date','cases','people_vaccinated','people_fully_vaccinated']].copy()

#Replace NaN Values with Zeros
df['people_vaccinated'] = df['people_vaccinated'].fillna(0)
df['people_fully_vaccinated'] = df['people_fully_vaccinated'].fillna(0)
df.tail(10)

In [ ]:
#df2 = df.loc[(df['date']>'2021-05-16') & (df['people_vaccinated'] == 0) ]

#remove last invalid rows 
df.drop(df[    (df['date']>'2021-05-16') & (df['people_vaccinated'] == 0)    ].index, inplace = True)
df.tail(10)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['date'], y=df['cases'], name="Cases"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['date'], y=df['people_vaccinated'], name = 'People vaccinated' ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=df['date'], y=df['people_fully_vaccinated'],name = 'People Fully vaccinated '),
    secondary_y= True,
)

fig.update_layout(
    autosize=False,
    width=700,
    height=500,
    title_text="Daily cases vs vaccination - "+country
)

fig.update_xaxes(title_text="Date")

fig.update_yaxes(title_text="Cases of covid", secondary_y=False)
fig.update_yaxes(title_text="Vaccinated", secondary_y=True)

fig.show()

In [ ]:
#reading dataset world population
dfpo= pd.read_csv("../input/population-by-country-2020/population_by_country_2020.csv")

#get Population of Israel 2020
dfpo = dfpo.loc[dfpo['Country (or dependency)']==country]
dfpo.head()

In [ ]:
#get in variable to convert
population= dfpo['Population (2020)'].values[0]
print(population)

In [ ]:
# calculate % of people vaccinated in selected country
df['% people_vaccinated'] = df['people_vaccinated']*100/population
df['% people_fully_vaccinated'] = df['people_fully_vaccinated']*100/population
df.head()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['date'], y=df['cases'], name="Cases"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['date'], y=df['% people_vaccinated'], name = '% People vaccinated' ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=df['date'], y=df['% people_fully_vaccinated'],name = '% People Fully vaccinated '),
    secondary_y= True,
)

fig.update_layout(
    autosize=False,
    width=700,
    height=500,
    title_text="Daily cases vs % vaccination - "+country
)

fig.update_xaxes(title_text="Date")

fig.update_yaxes(title_text="Cases of covid", secondary_y=False)
fig.update_yaxes(title_text="% Vaccinated", secondary_y=True)

fig.show()